In [1]:
# prep: import modules and get pwd
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass  # To get the password without showing the input
password = getpass.getpass()

········


In [2]:
# get the data
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)
query = '''SELECT
    f.film_id,
    f.title,
    f.release_year,
    f.language_id,
    f.rental_duration,
    f.rental_rate,
    f.length,
    f.rating,
    COUNT(CASE WHEN MONTH(r.rental_date) = 5 AND YEAR(r.rental_date) = 2005 THEN r.rental_id ELSE NULL END) AS rentals_count
FROM
    film AS f
LEFT JOIN
    inventory AS i ON f.film_id = i.film_id
LEFT JOIN
    rental AS r ON i.inventory_id = r.inventory_id
GROUP BY
    f.film_id
ORDER BY
    rentals_count DESC;'''



data = pd.read_sql_query(query, engine)
data.head(10)

,film_id,title,release_year,language_id,rental_duration,rental_rate,length,rating,rentals_count
0,103,BUCKET BROTHERHOOD,2006,1,7,4.99,133,PG,5
1,450,IDOLS SNATCHERS,2006,1,5,2.99,84,NC-17,5
2,735,ROBBERS JOON,2006,1,7,2.99,102,PG-13,5
3,86,BOOGIE AMELIE,2006,1,6,4.99,121,R,4
4,159,CLOSER BANG,2006,1,5,4.99,58,R,4
5,284,ENEMY ODDS,2006,1,5,4.99,77,NC-17,4
6,285,ENGLISH BULWORTH,2006,1,3,0.99,51,PG-13,4
7,303,FANTASY TROOPERS,2006,1,6,0.99,58,PG-13,4
8,305,FATAL HAUNTED,2006,1,6,2.99,91,PG,4
9,309,FEUD FROGMEN,2006,1,6,0.99,98,R,4


In [3]:
##2

In [4]:
# Define the SQL query
sql_query = """
SELECT
    f.title,
    CASE
        WHEN COUNT(r.rental_id) > 0 THEN 1
        ELSE 0
    END AS rented_in_may
FROM
    film f
LEFT JOIN
    inventory i ON f.film_id = i.film_id
LEFT JOIN
    rental r ON i.inventory_id = r.inventory_id AND MONTH(r.rental_date) = 5 AND YEAR(r.rental_date) = 2005
GROUP BY
    f.title
"""

# Execute the query and fetch the results into a DataFrame
df_rental_status = pd.read_sql(sql_query, engine)
df_rental_status

,title,rented_in_may
0,ACADEMY DINOSAUR,1
1,ACE GOLDFINGER,0
2,ADAPTATION HOLES,1
3,AFFAIR PREJUDICE,1
4,AFRICAN EGG,1
...,...,...
995,YOUNG LANGUAGE,0
996,YOUTH KICK,0
997,ZHIVAGO CORE,1
998,ZOOLANDER FICTION,1


In [5]:
# Concatenate the DataFrames along the rows
df = pd.merge(data, df_rental_status, on='title')
df= df.drop(["title"], axis=1)
df.head()

,film_id,release_year,language_id,rental_duration,rental_rate,length,rating,rentals_count,rented_in_may
0,103,2006,1,7,4.99,133,PG,5,1
1,450,2006,1,5,2.99,84,NC-17,5,1
2,735,2006,1,7,2.99,102,PG-13,5,1
3,86,2006,1,6,4.99,121,R,4,1
4,159,2006,1,5,4.99,58,R,4,1


In [6]:
#4
# Check for missing values
missing_values = df.isnull().sum().sum()
print("Missing Values:\n", missing_values)

# Data types of each column
data_types = df.dtypes
print("\nData Types:\n", data_types)

df.describe().T



Missing Values:
 0

Data Types:
 film_id              int64
release_year         int64
language_id          int64
rental_duration      int64
rental_rate        float64
length               int64
rating              object
rentals_count        int64
rented_in_may        int64
dtype: object


,count,mean,std,min,25%,50%,75%,max
film_id,1000.0,500.500,288.819436,1.00,250.75,500.50,750.25,1000.00
release_year,1000.0,2006.000,0.000000,2006.00,2006.00,2006.00,2006.00,2006.00
language_id,1000.0,1.000,0.000000,1.00,1.00,1.00,1.00,1.00
rental_duration,1000.0,4.985,1.411654,3.00,4.00,5.00,6.00,7.00
rental_rate,1000.0,2.980,1.646393,0.99,0.99,2.99,4.99,4.99
length,1000.0,115.272,40.426332,46.00,80.00,114.00,149.25,185.00
rentals_count,1000.0,1.156,1.053933,0.00,0.00,1.00,2.00,5.00
rented_in_may,1000.0,0.686,0.464349,0.00,0.00,1.00,1.00,1.00


In [7]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
# from sklearn.preprocessing import StandardScaler

X_num = df.select_dtypes(include = np.number)
X_cat = df.select_dtypes(include = object)

# Scaling data
transformer = MinMaxScaler().fit(X_num)
x_normalized = transformer.transform(X_num)
x_norm = pd.DataFrame(x_normalized, columns=X_num.columns)
x_norm

,film_id,release_year,language_id,rental_duration,rental_rate,length,rentals_count,rented_in_may
0,0.102102,0.0,0.0,1.00,1.0,0.625899,1.0,1.0
1,0.449449,0.0,0.0,0.50,0.5,0.273381,1.0,1.0
2,0.734735,0.0,0.0,1.00,0.5,0.402878,1.0,1.0
3,0.085085,0.0,0.0,0.75,1.0,0.539568,0.8,1.0
4,0.158158,0.0,0.0,0.50,1.0,0.086331,0.8,1.0
...,...,...,...,...,...,...,...,...
995,0.987988,0.0,0.0,1.00,0.5,0.669065,0.0,0.0
996,0.989990,0.0,0.0,0.00,0.0,0.899281,0.0,0.0
997,0.992993,0.0,0.0,0.75,0.5,0.949640,0.0,0.0
998,0.995996,0.0,0.0,0.75,0.0,0.985612,0.0,0.0


In [8]:
y = df['rented_in_may']
X = df.drop('rented_in_may', axis=1)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1337)

In [10]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
# from sklearn.preprocessing import StandardScaler

X_train_num = X_train.select_dtypes(include = np.number)

# Scaling data
transformer = MinMaxScaler().fit(X_train_num) # need to keep transformer
X_train_normalized = transformer.transform(X_train_num)
X_train_norm = pd.DataFrame(X_train_normalized, columns=X_train_num.columns)
X_train_norm

,film_id,release_year,language_id,rental_duration,rental_rate,length,rentals_count
0,0.140421,0.0,0.0,0.00,0.0,0.546763,0.6
1,0.334002,0.0,0.0,0.50,0.0,0.625899,0.0
2,0.105316,0.0,0.0,0.25,0.5,0.122302,0.0
3,0.773320,0.0,0.0,0.75,0.5,0.474820,0.4
4,0.099298,0.0,0.0,0.00,0.0,0.194245,0.6
...,...,...,...,...,...,...,...
795,0.277834,0.0,0.0,1.00,1.0,0.762590,0.4
796,0.543631,0.0,0.0,0.25,0.5,0.726619,0.4
797,0.557673,0.0,0.0,0.00,0.5,0.179856,0.0
798,0.355065,0.0,0.0,0.50,0.5,0.402878,0.4


In [11]:
X_train_categorical = X_train.select_dtypes(include = object)
X_train_cat = pd.get_dummies(X_train_categorical, 
                             columns=['rating'], drop_first=True
                            )

X_train_cat = X_train_cat.astype('int64')
X_train_cat.head(20)

,rating_NC-17,rating_PG,rating_PG-13,rating_R
46,0,1,0,0
789,0,0,1,0
722,0,0,0,0
283,1,0,0,0
39,0,0,0,1
350,0,1,0,0
652,0,0,0,0
377,1,0,0,0
547,0,1,0,0
805,0,1,0,0


In [12]:
X_train_transformed = np.concatenate([X_train_normalized, X_train_cat], axis=1)

In [18]:
X_train_transformed

array([[0.14042126, 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.33400201, 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.10531595, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.55767302, 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.3550652 , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ],
       [0.20060181, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [14]:
#5
from sklearn.linear_model import LogisticRegression
classification = LogisticRegression(random_state=0, solver='lbfgs',
                  multi_class='multinomial').fit(X_train_transformed, y_train)

In [15]:
classification = LogisticRegression(random_state=0, solver='saga',
                  multi_class='multinomial').fit(X_train_transformed, y_train)

In [16]:
# Can we now make predictions on the X_test?
predictions = classification.predict(X_test)
classification.score(X_test, y_test)


# NO - need to perform transformations on the X_test as well

C:\Users\beite\anaconda3\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


ValueError: could not convert string to float: 'R'